In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [2]:
# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
# load our saved model
#model.load('model.tflearn')

Training Step: 2999  | total loss: 0.21335 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.21335 - acc: 0.9864 -- iter: 16/24
Training Step: 3000  | total loss: 0.19583 | time: 0.034s
| Adam | epoch: 1000 | loss: 0.19583 - acc: 0.9877 -- iter: 24/24
--


In [3]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [4]:
p = bow("ماهي ساعات العمل اليوم", words)
print (p)

print(words)

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
['today', 'ابوابكم', 'اراك', 'استخراج', 'اقدم', 'الاوراق', 'التقديم', 'الحاجات', 'السلام', 'السلامة', 'الشهادة', 'العايزنها', 'العمل', 'المطلوبات', 'المطلوبة', 'اليوم', 'اليوم؟', 'ان', 'اي', 'تكلف', 'حالك', 'ساعات', 'ستفتحون', 'سعر', 'سعيد', 'سلام', 'شخص', 'شغالين', 'شكرا', 'عليكم', 'في', 'قروش', 'كان', 'كم', 'كيف', 'لاحقا', 'لقد', 'لك', 'للشهادة', 'للشهادة؟', 'ماهي', 'متى', 'متين', 'مرحبا', 'مع', 'مفيدا', 'هل', 'هنا؟', 'وقت', 'يمكن', 'يمكنني', 'يوجد', 'يوم']


In [5]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [6]:
classify('اشيل معاي ')

[('thanks', 0.6301116)]

In [7]:
response('اشيل اي الاوراق ')

صورة من الشهادة الثانوية وشهادة خلو طرف من القسم وايصال الدفع


In [8]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']))

            results.pop(0)

In [9]:
response('مرحبا')

'مرحبتين حبابك'

In [10]:
response('اجيب الاوراق')

'خلو طرف-صورة من شهادة الثانوي-ايصال الدفع'

In [ ]:
response('غبي')

In [ ]:
while True:
    ask = input('you: ')
    print('Neelain Bot: ', response(ask))

you: مرحبا
NeelainBot:  حبابك الف
you: عايز اقدم للشهادة
NeelainBot:  العفو
you: اقدم للشهادة
NeelainBot:  سعيد بمساعدتك
you: استخرج الشهادة
NeelainBot:  صورة من الشهادة الثانوية وشهادة خلو طرف من القسم وايصال الدفع
you: اين ادفع
NeelainBot:  سعيد بمساعدتك
you: مع السلامة
NeelainBot:  اتمنى لك يوما سعيدا
you: غبي
NeelainBot:  على الرحب والسعة :)
you: غبي
NeelainBot:  على الرحب والسعة :)
you: شكرا
NeelainBot:  على الرحب والسعة :)
you: حمار
NeelainBot:  على الرحب والسعة :)
you: ادفع وين
NeelainBot:  على الرحب والسعة :)
